In [111]:
import pandas as pd
import requests
import time
import numpy as np
from dotenv import load_dotenv
import os

In [77]:
dataset = pd.read_csv('X_reduced.csv', dtype={53: str, 54: str, 56: str})

In [79]:
dataset.head()

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Idle Mean,Idle Std,source_file,Flow ID,Src IP,Src Port,Dst IP
0,80,17,2018-02-21 10:08:51,119893441,309629,0,9908128,0,32,32,...,8,0.0,0.0,0.0,0.0,02-21-2018.csv,NaN,NaN,NaN,NaN
1,80,17,2018-02-21 10:08:52,119745803,259444,0,8302208,0,32,32,...,8,0.0,0.0,0.0,0.0,02-21-2018.csv,NaN,NaN,NaN,NaN
2,80,17,2018-02-21 10:08:53,119965025,248800,0,7961600,0,32,32,...,8,0.0,0.0,0.0,0.0,02-21-2018.csv,NaN,NaN,NaN,NaN
3,80,17,2018-02-21 10:08:54,119999991,272337,0,8714784,0,32,32,...,8,0.0,0.0,0.0,0.0,02-21-2018.csv,NaN,NaN,NaN,NaN
4,80,17,2018-02-21 10:08:54,119820553,219539,0,7025248,0,32,32,...,8,0.0,0.0,0.0,0.0,02-21-2018.csv,NaN,NaN,NaN,NaN


In [81]:
dataset['Src IP'].value_counts()

Src IP
18.218.55.126      307
52.14.136.135      306
18.218.229.235     305
18.216.24.42       304
18.216.200.189     304
18.219.5.43        303
18.218.11.51       302
18.219.9.1         302
18.219.32.43       299
18.218.115.60      268
8.6.0.1             36
172.31.69.25         7
131.202.242.193      4
52.224.180.249       2
122.226.181.165      2
66.198.240.13        1
115.238.245.8        1
84.73.10.63          1
200.241.132.151      1
12.156.73.33         1
94.231.103.172       1
37.97.217.235        1
190.214.77.75        1
107.180.108.2        1
91.185.191.213       1
221.194.47.233       1
221.194.47.239       1
122.226.181.166      1
60.191.38.77         1
185.171.50.102       1
Name: count, dtype: int64

In [29]:
dataset.columns

Index(['Dst Port', 'Protocol', 'Timestamp', 'Flow Duration', 'Tot Fwd Pkts',
       'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max',
       'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std',
       'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean',
       'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean',
       'Flow IAT Std', 'Fwd IAT Tot', 'Fwd IAT Mean', 'Bwd IAT Tot',
       'Bwd IAT Mean', 'Bwd IAT Min', 'Fwd PSH Flags', 'Fwd URG Flags',
       'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s',
       'Pkt Len Mean', 'Pkt Len Std', 'SYN Flag Cnt', 'RST Flag Cnt',
       'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt', 'CWE Flag Count',
       'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg', 'Fwd Seg Size Avg',
       'Bwd Seg Size Avg', 'Init Fwd Win Byts', 'Init Bwd Win Byts',
       'Fwd Act Data Pkts', 'Fwd Seg Size Min', 'Active Mean', 'Active Std',
       'Idle Mean', 'Idle Std', 'source_file', 'Flow ID', 'Src 

In [23]:
dotenv_path = os.getenv('DOTENV_PATH')
if dotenv_path:
    load_dotenv(dotenv_path)

API_KEY = os.getenv('ABUSEIPDB_API_KEY')

headers = {
    'Accept': 'application/json',
    'Key': API_KEY
}

def consultar_ip(ip):
    url = 'https://api.abuseipdb.com/api/v2/check'
    params = {
        'ipAddress': ip,
        'maxAgeInDays': 90
    }

    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()['data']
        return data['abuseConfidenceScore']
    except Exception as e:
        print(f"Error con IP {ip}: {e}")
        return None

In [103]:
nan_src_rows = dataset[dataset['Src IP'].isna()].copy()
n_insertar = int(0.3 * len(nan_src_rows)) # Número de filas a modificar (30% del dataset)
filas_a_modificar = nan_src_rows.sample(n=n_insertar, random_state=42)
ips_random = np.random.choice(ips_maliciosas, size=n_insertar)
dataset.loc[filas_a_modificar.index, 'Src IP'] = ips_random

In [89]:
ips_unicas = dataset['Src IP'].dropna().unique()
print(len(ips_unicas))

50


In [85]:
ips_maliciosas = ["64.62.156.110",
                  "93.174.95.106",
                  "190.117.78.191",
                  "20.65.195.46",
                  "64.62.156.61",
                  "64.62.156.98",
                  "193.163.125.250",
                  "193.163.125.77",
                  "193.163.125.54",
                  "47.250.80.223",
                  "35.203.210.165",
                  "60.13.138.212",
                  "41.76.247.3",
                  "147.185.133.109",
                  "103.29.69.96",
                  "147.185.133.153",
                  "188.209.244.27",
                  "162.142.125.134",
                  "163.47.37.130",
                  "162.216.150.15"]

In [93]:
ips_unicas = dataset['Src IP'].dropna().unique()

ip_scores = {} # Crear un diccionario cacheado para no consultar la misma IP varias veces
for ip in ips_unicas:
    score = consultar_ip(ip)
    ip_scores[ip] = score
    time.sleep(1)  # pausa para no exceder el límite de la API

dataset['src_abuse_score'] = dataset['Src IP'].map(ip_scores) # mapeo

In [94]:
dataset['src_abuse_score'].value_counts()

src_abuse_score
0.0      4573
100.0    3708
13.0      327
27.0      321
5.0       314
Name: count, dtype: int64

In [97]:
ip_scores

{'60.13.138.212': 13,
 '162.216.150.15': 0,
 '20.65.195.46': 100,
 '35.203.210.165': 0,
 '147.185.133.109': 0,
 '188.209.244.27': 5,
 '64.62.156.61': 100,
 '47.250.80.223': 100,
 '193.163.125.250': 100,
 '93.174.95.106': 100,
 '64.62.156.110': 100,
 '147.185.133.153': 0,
 '190.117.78.191': 100,
 '193.163.125.77': 100,
 '41.76.247.3': 27,
 '103.29.69.96': 100,
 '162.142.125.134': 0,
 '193.163.125.54': 100,
 '163.47.37.130': 100,
 '64.62.156.98': 100,
 '52.224.180.249': 0,
 '8.6.0.1': 0,
 '60.191.38.77': 0,
 '172.31.69.25': 0,
 '122.226.181.166': 0,
 '221.194.47.239': 0,
 '122.226.181.165': 0,
 '221.194.47.233': 0,
 '131.202.242.193': 0,
 '91.185.191.213': 0,
 '107.180.108.2': 0,
 '190.214.77.75': 0,
 '37.97.217.235': 0,
 '115.238.245.8': 0,
 '66.198.240.13': 0,
 '185.171.50.102': 0,
 '84.73.10.63': 0,
 '200.241.132.151': 0,
 '12.156.73.33': 0,
 '94.231.103.172': 0,
 '52.14.136.135': 0,
 '18.219.9.1': 0,
 '18.218.229.235': 0,
 '18.216.200.189': 0,
 '18.218.55.126': 0,
 '18.218.11.51': 0,